In [2]:
import os
import tensorflow as tf


In [3]:
# forming the text data pipeline using tf.data.Dataset

def get_filenames(base_dir):
    pos =[]
    neg =[]
    paths =[]
    for folder,sub,files in os.walk(os.path.join(base_dir
                                                )):
        print('Currently looking in :',folder)
        if folder.split(os.path.sep)[-1] =='pos':
            print('Getting files paths for positive reviews ...')
            for f_name in files:
                paths.append(os.path.join(folder,f_name))
                pos.append(os.path.join(folder,f_name))
        elif folder.split(os.path.sep)[-1] =='neg':
            print('Getting file paths for negative reviews...')
            for f_name in files:
                paths.append(os.path.join(folder,f_name))
                neg.append(os.path.join(folder,f_name))
        else :
            pass
        
    return paths
            
        
    
                                 

In [4]:
base_dir='/Users/kakhan/Desktop/ML_Practise/aclImdb'
filenames_train = get_filenames(os.path.join(base_dir,'train'))
filenames_test = get_filenames(os.path.join(base_dir,'test'))

Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/train
Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/train/neg
Getting file paths for negative reviews...
Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/train/pos
Getting files paths for positive reviews ...
Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/test
Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/test/neg
Getting file paths for negative reviews...
Currently looking in : /Users/kakhan/Desktop/ML_Practise/aclImdb/test/pos
Getting files paths for positive reviews ...


In [5]:
print(len(filenames_train))
print(len(filenames_test))

25000
25000


In [6]:
# preparing the tf.data.Dataset
batch_size=32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    os.path.join(base_dir,'train'),
    batch_size=32,
    seed=42,
    validation_split=0.2,
    subset='training'
    )

class_names = raw_train_ds.class_names # the train dir has 2 sub directories ['pos' and 'neg ']
# so it uses those names for the class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    os.path.join(base_dir,'test'),
    batch_size=32

).cache().shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

#use the same training dir to form the validation dataset by using the subset='validation' 
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    os.path.join(base_dir,'train'),
    batch_size=32,
    seed=42,
    validation_split=0.2,
    subset='validation'
    ).cache().prefetch(tf.data.experimental.AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [6]:
for text_batch , label_batch in train_ds.take(1):
    print(label_batch)

tf.Tensor([0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1], shape=(32,), dtype=int32)


In [8]:
os.getcwd()

'/Users/kakhan/Desktop/ML_Practise'

In [13]:
import tempfile
path1 = os.path.join(tempfile.gettempdir(), "train_ds")
path2 = os.path.join(tempfile.gettempdir(), "val_ds")
path3 = os.path.join(tempfile.gettempdir(), "test_ds")

tf.data.experimental.save(train_ds,path='/Users/kakhan/Desktop/ML_Practise/imdb_tf_data/train_ds')
tf.data.experimental.save(test_ds,path='/Users/kakhan/Desktop/ML_Practise/imdb_tf_data/test_ds')
tf.data.experimental.save(val_ds,path='/Users/kakhan/Desktop/ML_Practise/imdb_tf_data/val_ds')

In [14]:
new_dataset = tf.data.experimental.load(path='/Users/kakhan/Desktop/ML_Practise/imdb_tf_data/train_ds')

In [16]:
for text,label in new_dataset.take(1):
    
    print(label)

tf.Tensor([0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1], shape=(32,), dtype=int32)


In [8]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

bert_model_url = map_name_to_handle[bert_model_name]
preprocessor_url = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {bert_model_url}')
print(f'Preprocess model auto-selected: {preprocessor_url}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [9]:
# downloading the bert model and the preprocess model from tensorflow hub
import tensorflow_hub as hub
import tensorflow_text as text



In [11]:
#bert layers has a max_seq_length of 128
# all other are truncated or padded with zeros
bert_layer = hub.KerasLayer(bert_model_url,trainable=True)


In [25]:
bert_preprocessor_layer = hub.KerasLayer(preprocessor_url,name='preprocessor')

In [26]:
sample_text = ['Hi my name is Farhan','I am practising Deep learning']
preprocessed_texts = bert_preprocessor_layer(sample_text)

In [36]:
bert_outputs = bert_layer(preprocessed_texts)

In [50]:
bert_outputs['pooled_output']

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.89338595, -0.564671  , -0.8697556 , ..., -0.7986418 ,
        -0.7899533 ,  0.91862684],
       [-0.81916755, -0.27288994,  0.32206187, ...,  0.01590588,
        -0.5365835 ,  0.888802  ]], dtype=float32)>

In [57]:
# creating the model
def create_model():
    text_input = tf.keras.Input(shape=(),dtype=tf.string,name='text')
    preprocessed_text_encodings = bert_preprocessor_layer(text_input)
    bert_layer_output = bert_layer(preprocessed_text_encodings)
    #for fine tuning we'll use the pooled ouputs 
    pooled_output = bert_layer_output['pooled_output']
    dropout = tf.keras.layers.Dropout(0.2)(pooled_output)
    output = tf.keras.layers.Dense(1,activation=None,name='output')(dropout)
    return tf.keras.Model(text_input,output)
    

In [66]:
model = create_model()
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessor (KerasLayer)       {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'default': (None, 7 109482241   preprocessor[5][0]               
                                                                 preprocessor[5][1]               
                                                                 preprocessor[5][2]               
____________________________________________________________________________________________

In [67]:
model(tf.constant(['this movie was great'])).numpy()

array([[0.48559433]], dtype=float32)